In [1]:
from pulp import *

In [2]:
probA=LpProblem("Workforce Planning",LpMinimize)
month=[1,2,3,4,5,6]
demand={
1:100,
2:100,
3:115,
4:125,
5:140,
6:150   
}

turnover= 0.05
succ_comp = 0.9
prod_inc = 0.2
sal_untr = 3000
sal_tr = 3300
initial_untr =145
initial_trained =0

untrained_vars = LpVariable.dicts("u", month,lowBound=0, cat='Contiuous')
layoff_vars = LpVariable.dicts("l", month,lowBound=0, cat='Contiuous')
start_train_vars =LpVariable.dicts("x", month,lowBound=0, cat='Contiuous')
trained_vars =LpVariable.dicts("tr", month,lowBound=0, cat='Contiuous')

In [3]:
probA += sal_untr*lpSum([untrained_vars[i]-layoff_vars[i] for i in month]) + \
sal_tr*lpSum([trained_vars[i] for i in month]) , "total salary"

# initial condition
probA += untrained_vars[1]==initial_untr,"initial untrained"
probA += trained_vars[1]==initial_trained,"initial trained"

for i in month[:len(month)-1]:
      # untrained worker blance between months:
    probA += (1-turnover)*(untrained_vars[i]-layoff_vars[i]-start_train_vars[i])\
    +(1-succ_comp)*start_train_vars[i] == untrained_vars[i+1],"untrained_balance_%d"%i
    # trained worker blance between months:
    probA += trained_vars[i]+succ_comp*start_train_vars[i] == trained_vars[i+1],"trained_balance_%d"%i
    #satsify demand in month i
    probA += (untrained_vars[i]-layoff_vars[i]-start_train_vars[i])\
    +(1+prod_inc)*trained_vars[i] >= demand[i],"satisfy demand_%d"%i
    #layoff limit
    probA += layoff_vars[i] <= untrained_vars[i],"layoff limit_%d"%i
    #traininng limit
    probA += start_train_vars[i]<= untrained_vars[i]-layoff_vars[i] ,"training limit_%d"%i

#satsify demand in month 6
probA +=(untrained_vars[len(month)]-layoff_vars[len(month)]-start_train_vars[len(month)])\
    +(1+prod_inc)*trained_vars[len(month)] >= demand[len(month)],"satisfy demand_%d"%len(month)
#layoff limit
probA += layoff_vars[len(month)] <= untrained_vars[len(month)],"layoff limit_%d"%len(month)
#traininng limit
probA += start_train_vars[len(month)]<= untrained_vars[len(month)]-layoff_vars[len(month)],"training limit_%d"%len(month)


i= 1
i= 2
i= 3
i= 4
i= 5


In [9]:
probA.writeLP("Workforce.lp")
probA.solve()
print("Status:",LpStatus[probA.status])

Status: Optimal


In [10]:
for v in probA.variables():
    print(v.name, "=", v.varValue,"\tReduced Cost =", v.dj)
print("Total cost =", value(probA.objective))
original_cost=value(probA.objective)

l_1 = 8.113002 	Reduced Cost = 0.0
l_2 = 0.0 	Reduced Cost = 733.79872
l_3 = 0.0 	Reduced Cost = 1660.4629
l_4 = 0.0 	Reduced Cost = 2802.8511
l_5 = 0.0 	Reduced Cost = 4191.6097
l_6 = 0.0 	Reduced Cost = 5865.771
tr_1 = 0.0 	Reduced Cost = -4.5474735e-13
tr_2 = 33.198298 	Reduced Cost = 0.0
tr_3 = 67.87229 	Reduced Cost = 0.0
tr_4 = 92.580309 	Reduced Cost = -1.3642421e-12
tr_5 = 111.22587 	Reduced Cost = 4.5474735e-13
tr_6 = 119.10198 	Reduced Cost = 0.0
u_1 = 145.0 	Reduced Cost = 0.0
u_2 = 98.6887 	Reduced Cost = 4.5474735e-13
u_3 = 61.006606 	Reduced Cost = 2.2737368e-12
u_4 = 34.620925 	Reduced Cost = 2.2737368e-13
u_5 = 15.280178 	Reduced Cost = 1.1368684e-12
u_6 = 7.077626 	Reduced Cost = 0.0
x_1 = 36.886998 	Reduced Cost = -9.094947e-13
x_2 = 38.526658 	Reduced Cost = -4.5474735e-13
x_3 = 27.453354 	Reduced Cost = -1.3642421e-12
x_4 = 20.717295 	Reduced Cost = 7.0485839e-12
x_5 = 8.7512267 	Reduced Cost = 6.8212103e-13
x_6 = 0.0 	Reduced Cost = 8865.771
Total cost = 2459812.96